# 利用用户行为数据
如何了解一个人呢？

**通过用户留下的文字和行为了解用户兴趣和需求。**


实现个性化推荐的最理想情况是用户在注册的时候主动告知其喜欢什么。

3个缺点：

1.  现在的自然语言理解技术很难理解用户用来描述兴趣的自然语言；
2.  用户的兴趣是不断变化的；
3.  很多时候用户并不知道自己喜欢什么，或者很难用语言描述。


**基于用户行为分析的推荐算法是个性化推荐系统的重要算法，学术界一般将这种类型的算法称为协同过滤<sup><a id="fnr.1" class="footref" href="#fn.1">1</a></sup>算法。**
**用户行为数据在网站上最简单的存在形式就是日志。**

-   原始日志（raw log）
-   会话日志（session log）

    将多种原始日志按照用户行为汇总，每个会话表示一次用户行为和对应的服务。

-   展示日志（impression log）

    搜索引擎和搜索广告系统服务为每次查询生成的，记录了查询和返回结果。

-   点击日志（click log）

    用户点击了某个返回结果的点击信息。

一个并行程序会周期性地归并展示日志和点击日志，得到的会话日志中每个消息是一个用户提交的查询、得到的结果以及点击。

推荐系统和电子商务网站也会汇总原始日志生成描述用户行为的会话日志。会话日志通常存储在分布式数据仓库中，这些日志记录了用户的各种行为。


用户行为在个性化推荐系统中一般分为两种：
-   **显性反馈行为（explicit feedback）:** 用户明确表示对物品喜好的行为。主要方式就是 **评分** 和 **喜欢/不喜欢** 。

-   **隐性反馈行为（implicit feedback）:** 不能明确反应用户喜好的行为。最具代表性的就是 **页面浏览行为** 。

|          | 显性反馈数据 | 隐性反馈数据   |
|----------|--------------|----------------|
| 用户兴趣 | 明确         | 不明确         |
| 数量     | 较少         | 庞大           |
| 存储     | 数据库       | 分布式文件系统 |
| 实时读取 | 实时         | 有延迟         |
| 正负反馈 | 都有         | 只有正反馈     |


按照反馈的方向分，可分为：
-   **正反馈:** 用户的行为倾向于指用户喜欢该物品。
-   **负反馈:** 用户的行为倾向于指用户不喜欢该物品。

在显性反馈中，很容易区分一个用户行为是正反馈还是负反馈，在隐性反馈行为中，相对比较难以确定。

|              | 显性反馈                   | 隐性反馈                               |
|--------------|----------------------------|----------------------------------------|
| 视频网站     | 用户对视频的评分           | 用户观看视频的日志、浏览视频页面的日志 |
| 电子商务网站 | 用户对商品的评分           | 购买日志、浏览日志                     |
| 门户网站     | 用户对新闻的评分           | 阅读新闻的日志                         |
| 音乐网站     | 用户对音乐/歌手/专辑的评分 | 听歌的日志                             |


一种统一的表示用户行为的方式，它将一个用户行为表示为6部分，即产生行为的用户和行为的对象、行为的种类、产生行为的上下文、行为的内容和权重。

| 类型             | 标识             | 解释                                                                                               |
|------------------|------------------|----------------------------------------------------------------------------------------------------|
| 产生行为的用户   | user id          | 产生行为的用户的唯一标识                                                                           |
| 行为的对象       | item id          | 产生行为的对象的唯一标识                                                                           |
| 行为的种类       | behavior type    | 行为的种类（比如是购买还是浏览）                                                                   |
| 产生行为的上下文 | context          | 产生行为的上下文，包括时间和地点等                                                                 |
| 行为的权重       | behavior weight  | 行为的权重（如果是观看视频的行为，那么这个权重可以是观看时长；如果是打分行为，这个权重可以是分数） |
| 行为的内容       | behavior content | 行为的内容（如果是评论行为，那么就是评论的文本；如果是打标签的行为，就是标签）                     |

不同的数据集包含不同的行为，目前比较有代表性的数据集有下面几个。
-   **无上下文信息的隐性反馈数据集**

    每一条行为记录仅仅包含用户ID和物品ID。

    Book-Crossing数据集。<sup><a id="fnr.2" class="footref" href="#fn.2">2</a></sup>

-   无上下文信息的显性反馈数据集

    每一条记录包含用户ID、物品ID和用户对物品的评分。

-   有上下文信息的隐性反馈数据集

    每一条记录包含用户ID、物品ID和用户对物品产生行为的时间戳。

    Lastfm数据集。<sup><a id="fnr.3" class="footref" href="#fn.3">3</a></sup>

-   有上下文信息的显性反馈数据集

    每一条记录包含用户ID、物品ID、用户对物品的评分和评分行为发生的时间戳。

    Netflix Prize数据集。<sup><a id="fnr.4" class="footref" href="#fn.4">4</a></sup>



<a id="org7a972d7"></a>
## 用户行为分析

本节将介绍用户行为数据中蕴含的一般规律，这些规律是存在于很多网站中的普遍规律。


<a id="org4a0a610"></a>

### 用户活跃度和物品流行度的分布

互联网上的很多数据分布都满足一种称为Power Law的分布，在互联网领域也称长尾分布。

用户行为数据也蕴含这种规律。令 $f_u(k)$ 为对 $k$ 个物品产生过行为的用户数，令 $f_i(k)$ 为被 $k$ 个用户产生过行为的物品数， $f_u(k)$ 和 $f_i(k)$ 都满足长尾分布：

\begin{equation}
\begin{split}
& f_i(k) = \alpha_i k^{\beta_i} \\
& f_u(k) = \alpha_u k^{\beta_u} \\
\end{split} \nonumber
\end{equation}

可选择 Delicious 和 CiteULike 数据集进行分析。



<a id="org4056928"></a>
### 用户活跃度和物品流行度的关系

一般认为，新用户倾向于浏览热门的物品，因其对网站还不熟悉，而老用户会逐渐开始浏览冷门的物品。

**用户越活跃，越倾向于浏览冷门的物品。**



<a id="org5862f10"></a>
## 协同过滤算法

仅仅基于用户行为数据设计的推荐算法一般称为协同过滤算法。

协同过滤算法中有很多方法，比如基于邻域的方法（neighborhood-based）、隐语义模型（latent factor model）等。


<a id="orga1c1e8a"></a>
### 基于邻域（neighborhood-based）的算法

最著名、在业界得到最广泛应用的算法是基于邻域的方法，主要包括：

-   **基于用户的协同过滤算法:** 给用户推荐和其兴趣相似的其他用户喜欢的物品。
-   **基于物品的协同过滤算法:** 给用户推荐和其之前喜欢的物品相似的物品。



<a id="orga0cf053"></a>
### 实验设计和算法评测

本节将通过离线实验方法评测提到的算法。

首先介绍用到的数据集，然后介绍采用的实验方法和评测指标。



<a id="orga702f41"></a>
#### 数据集
本章采用GroupLens提供的MovieLens数据集介绍和评测各种算法。 MovieLens数据集有3个不同的版本，本章选用中等大小的数据集。该数据集包含6000多用户对4000多部电影的100万条评分。该数据集是一个评分数据集，用户可以给电影评5个不同等级的分数(1~5分)。本章着重研究隐反馈数据集中的TopN推荐问题，因此忽略了数据集中的评分记录。也就是说，TopN推荐的任务是预测用户会不会对某部电影评分，而不是预测用户在准备对某部电影评分的前提下会给电影评多少分。

加载数据：


In [ ]:
def load_movielens(path='./movielens/ml-100k'):
    # get movie titles
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = title
    # load data
    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs

In [ ]:
prefs = load_movielens()
print(prefs['87'])

<a id="org8ba2fe4"></a>
#### 实验设计

协同过滤算法的离线实验一般如下设计。

1.  将用户行为数据集按照均匀分布随机分成 $M$ 份(例如取 $M=8$ )，挑选一份作为测试集，将剩下的 $M-1$ 份作为训练集。
2.  在训练集上建立用户兴趣模型，并在测试集上对用户行为进行预测，统计出相应的评测指标。为了保证评测指标并不是过拟合的结果，需要进行 $M$ 次实验，并且每次都使用不同的测试集。
3.  将 $M$ 次实验测出的评测指标的平均值作为最终的评测指标。

下面的代码描述了将数据集随机分成训练集和测试集的过程：

```python
def split_data(data, M, k, seed):
    test = []
    train = []
    random.seed(seed)
    for user, item in data:
        if random.randint(0, M) == k:
            test.append([user, item])
        else:
            train.append([user, item])
    return train, test
```

每次实验选取不同的 $k(0≤k≤M-1)$ 和相同的随机数种子seed，进行 $M$ 次实验就可以得到 $M$ 个不同的训练集和测试集，然后分别进行实验，用 $M$ 次实验的平均值作为最后的评测指标。这样做主要是防止某次实验的结果是过拟合的结果(over fitting)，但如果数据集够大，模型够简单，为了快速通过离线实验初步地选择算法，也可以只进行一次实验。



<a id="org1e73bfe"></a>
#### 评测指标

对用户 $u$ 推荐 $N$ 个物品（记为 $R(u)$ ），令用户 $u$ 在测试集上喜欢的物品集合为 $T(u)$ ，然后可以通过准确率/召回率评测推荐算法的精度:

\begin{equation}
\begin{split}
& Recall = \frac{\sum_u |R(u) \cap T(u)|}{\sum_u |T(u)|} \\
& Precision = \frac{\sum_u |R(u) \cap T(u)|}{\sum_u |R(u)|} \\
\end{split} \nonumber
\end{equation}

召回率描述有多少比例的用户——物品评分记录包含在最终的推荐列表中，而准确率描述最终的推荐列表中有多少比例是发生过的用户——物品评分记录。

下面两段代码给出了召回率和准确率的计算方法。

```python
def recall(train, test, N, GetRecommendation=None):
    hit = 0
    all = 0
    for user in train.keys():
        tu = test[user]
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += len(tu)
    return hit / (all * 1.0)
```

```python
def precision(train, test, N, GetRecommendation=None):
    hit = 0
    all = 0
    for user in train.keys():
        tu = test[user]
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            if item in tu:
                hit += 1
        all += N
    return hit / (all * 1.0)
```

下面的公式给出了最简单的覆盖率定义，覆盖率反映了推荐算法发掘长尾的能力，覆盖率越高，说明推荐算法越能够将长尾中的物品推荐给用户。

\begin{equation}
Coverage = \frac{U_{u \in U} R(u)}{|I|} \nonumber
\end{equation}

该覆盖率表示最终的推荐列表中包含多大比例的物品。如果所有的物品都被推荐给至少一个用户，那么覆盖率就是100%。如下代码可以用来计算推荐算法的覆盖率：

```python
def coverage(train, test, N, GetRecommendation=None):
    recommend_items = set()
    all_items = set()
    for user in train.keys():
        for item in train[user].keys():
            all_items.add(item)
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            recommend_items.add(item)
    return len(recommend_items) / (len(all_items) * 1.0)
```

最后，用推荐列表中物品的平均流行度度量推荐结果的新颖度。如果推荐出的物品都很热门，说明推荐的新颖度较低，否则说明推荐结果比较新颖。

```python
def popularity(train, test, N, GetRecommendation=None):
    item_popularity = dict()
    for user, items in train.items():
        for item in items.keys():
            if item not in item_popularity:
                item_popularity[item] = 0
            item_popularity[item] += 1
    ret = 0
    n = 0
    for user in train.keys():
        rank = GetRecommendation(user, N)
        for item, pui in rank:
            ret += math.log(1 + item_popularity[item])
            n += 1
    ret /= n * 1.0
    return ret
```

在计算平均流行度时对每个物品的流行度取对数，这是因为物品的流行度分布满足长尾分布，在取对数后，流行度的平均值更加稳定。


## 脚注

<sup><a id="fn.1" class="footnum" href="#fnr.1">1</a></sup> 协同过滤就是指用户可以齐心协力，通过不断地和网站互动，使自己的推荐列表能够不断过滤掉自己不感兴趣的物品，从而越来越满足自己的需求。

<sup><a id="fn.2" class="footnum" href="#fnr.2">2</a></sup> 参见“Book-Crossing Dataset”，地址为<http://www.informatik.uni-freiburg.de/~cziegler/BX/>。

<sup><a id="fn.3" class="footnum" href="#fnr.3">3</a></sup> 参见<http://www.dtic.upf.edu/~ocelma/MusicRecommendationDataset/lastfm-1K.html>。

<sup><a id="fn.4" class="footnum" href="#fnr.4">4</a></sup> 参见<http://netflixprize.com/>。